# Project 2: PQC Lattice-Based Kyber (Real Library Demo)

This notebook demonstrates the **CRYSTALS-Kyber** KEM. We are using the "Noisy Lockbox" 📦 analogy.

* **Bob (The Receiver):** Creates a "magic lock" (public key 🔒) and a "magic key" (private key 🗝️).
* **Alice (The Sender):** Uses Bob's lock 🔒 to create a new secret 💎 and a "locked box" (`ciphertext`) 📦 containing it.
* **Bob:** Uses his key 🗝️ to open the box 📦 and get the secret 💎.

# Pre Requisites (run these commands first)

* `git clone --depth=1 https://github.com/open-quantum-safe/liboqs-python`
* `cd/liboqs-python`
* `pip install .`
* `export PYTHONPATH=$PYTHONPATH:/path/to/liboqs-python`


In [2]:
import oqs
import logging
from pprint import pformat
from sys import stdout

logger = logging.getLogger("KyberDemo")
logger.setLevel(logging.INFO)
logger.handlers.clear()
logger.addHandler(logging.StreamHandler(stdout))

logger.info("1. IMPORTS AND SETUP")
logger.info("liboqs version: %s", oqs.oqs_version())
logger.info("liboqs-python version: %s", oqs.oqs_python_version())
logger.info(
    "Enabled KEM mechanisms:\n%s",
    pformat(oqs.get_enabled_kem_mechanisms(), compact=True),
)

1. IMPORTS AND SETUP
liboqs version: 0.14.0
liboqs-python version: 0.14.0
Enabled KEM mechanisms:
('BIKE-L1', 'BIKE-L3', 'BIKE-L5', 'Classic-McEliece-348864',
 'Classic-McEliece-348864f', 'Classic-McEliece-460896',
 'Classic-McEliece-460896f', 'Classic-McEliece-6688128',
 'Classic-McEliece-6688128f', 'Classic-McEliece-6960119',
 'Classic-McEliece-6960119f', 'Classic-McEliece-8192128',
 'Classic-McEliece-8192128f', 'Kyber512', 'Kyber768', 'Kyber1024', 'ML-KEM-512',
 'ML-KEM-768', 'ML-KEM-1024', 'sntrup761', 'FrodoKEM-640-AES',
 'FrodoKEM-640-SHAKE', 'FrodoKEM-976-AES', 'FrodoKEM-976-SHAKE',
 'FrodoKEM-1344-AES', 'FrodoKEM-1344-SHAKE')


## Step 2: Bob's Side (The Receiver)

We begin by setting up Bob. He is the **receiver**. He needs to generate his key pair.

* **Public Key 🔒:** The "magic lock" he will give to Alice.
* **Private Key 🗝️:** The "magic key" he keeps secret.

In [3]:
logger.info("\n2. BOB'S SIDE (THE RECEIVER)")

# We select the KEM algorithm
kemalg = "Kyber512"

# Bob creates his KEM object
kem_bob = oqs.KeyEncapsulation(kemalg)

# Bob generates his key pair
logger.info("Bob (Receiver) is generating his key pair...")
public_key_bob = kem_bob.generate_keypair()

logger.info("Bob has his public key 🔒 and private key 🗝️.")
logger.info("Bob's Public Key (first 30 bytes):\n%s...", public_key_bob.hex()[:30])


2. BOB'S SIDE (THE RECEIVER)
Bob (Receiver) is generating his key pair...
Bob has his public key 🔒 and private key 🗝️.
Bob's Public Key (first 30 bytes):
6af70d815160cee9c4c03839d5db94...


## Step 3: Alice's Side (The Sender)

Now, Alice (the **sender**) gets Bob's public key 🔒.

Her goal is to "encapsulate" a new secret. The `.encap_secret()` function is magic:
1.  It creates a new, random secret 💎 (`shared_secret_alice`).
2.  It uses Bob's lock 🔒 to create a "locked box" 📦 (`ciphertext`).
3.  It gives *both* back to Alice.

In [4]:
logger.info("\n3. ALICE'S SIDE (THE SENDER)")

# Alice creates her own KEM object
kem_alice = oqs.KeyEncapsulation(kemalg)
logger.info("Alice has Bob's public key 🔒.")

# Alice runs the "Encapsulation" step
logger.info("Alice is creating a new secret and locking it...")
ciphertext, shared_secret_alice = kem_alice.encap_secret(public_key_bob)

logger.info("Alice has her new secret (for herself).")
logger.info("Alice's Secret (first 30 bytes): \n%s...", shared_secret_alice.hex()[:30])
logger.info("\nAlice sends the 'locked box' 📦 to Bob.")
logger.info("Ciphertext (first 30 bytes): \n%s...", ciphertext.hex()[:30])


3. ALICE'S SIDE (THE SENDER)
Alice has Bob's public key 🔒.
Alice is creating a new secret and locking it...
Alice has her new secret (for herself).
Alice's Secret (first 30 bytes): 
8ab452be58d75ad4aa1bb5cdb3457a...

Alice sends the 'locked box' 📦 to Bob.
Ciphertext (first 30 bytes): 
d119dc9bc394e75f754d7e007d0200...


## Step 4: Bob's Side (The "Decapsulation")

Bob receives the `ciphertext` 📦 from Alice. He now uses his **private key 🗝️** (which his `kem_bob` object has been holding) to "decapsulate" (unlock) the box.

In [5]:
logger.info("\n4. BOB'S SIDE (THE DECAPSULATION)")
logger.info("Bob receives the 'locked box' 📦.")
logger.info("Bob uses his private key 🗝️ to open it...")

# Bob uses his KEM object (which has his private key)
# to decrypt the ciphertext.
shared_secret_bob = kem_bob.decap_secret(ciphertext)

logger.info("Bob has retrieved the secret from the box.")
logger.info("Bob's Retrieved Secret (first 30 bytes): \n%s...", shared_secret_bob.hex()[:30])


4. BOB'S SIDE (THE DECAPSULATION)
Bob receives the 'locked box' 📦.
Bob uses his private key 🗝️ to open it...
Bob has retrieved the secret from the box.
Bob's Retrieved Secret (first 30 bytes): 
8ab452be58d75ad4aa1bb5cdb3457a...


## Step 5: The Test Case (The Proof)

This is the "Aha!" moment. We check if the secret Alice *created* is identical to the secret Bob *retrieved*.

In [6]:
logger.info("\n5. THE TEST CASE")

# We compare the two secrets
secrets_match = (shared_secret_alice == shared_secret_bob)

logger.info("Does Alice's secret == Bob's secret?")
logger.info("Shared secrets coincide: %s", secrets_match)

if secrets_match:
    logger.info("   >>> SUCCESS! <<<")
    logger.info("   Alice has successfully sent a quantum-resistant secret to Bob.")
else:
    logger.info("   >>> FAILURE! <<<")
    
print("\n\nEnd of PQC-Kyber Simulation")


5. THE TEST CASE
Does Alice's secret == Bob's secret?
Shared secrets coincide: True
   >>> SUCCESS! <<<
   Alice has successfully sent a quantum-resistant secret to Bob.


End of PQC-Kyber Simulation
